In [11]:
import numpy as np

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.cmv
posts_collection = db.posts
tl_comments_collection = db.tl_comments
deltad_replies_collection = db.deltad_replies

In [8]:
deltad_post_gen = posts_collection.find( {'deltad_tl_comments': {"$exists": True}})
undeltad_post_gen = posts_collection.find( {'deltad_tl_comments': {"$exists": False}})

In [9]:
deltad_post_list = list(deltad_post_gen)
undeltad_post_list = list(undeltad_post_gen)

In [10]:
print(len(deltad_post_list))
print(len(undeltad_post_list))

1537
7100


In [13]:
np.random.shuffle(deltad_post_list)
np.random.shuffle(undeltad_post_list)

In [17]:
#train test split

train_posts = []
test_posts = []

test_split_d = int(0.2*len(deltad_post_list))
test_split_u = int(0.2*len(undeltad_post_list))

test_posts.extend(deltad_post_list[0:test_split_d])
test_posts.extend(undeltad_post_list[0:test_split_u])

train_posts.extend(deltad_post_list[test_split_d::])
train_posts.extend(undeltad_post_list[test_split_u::])

In [18]:
len(train_posts)

6910

In [19]:
len(test_posts)

1727

In [20]:
train_post_texts = [doc['post_text'] for doc in train_posts]
test_post_texts = [doc['post_text'] for doc in test_posts]

In [88]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
import string

In [76]:
def strip_punctuation_digits(text):
    to_replace = [punc for punc in string.punctuation+string.digits]
    replacement_table = str.maketrans({key: None for key in to_replace})
    stripped_text = text.translate(replacement_table)
    return stripped_text
                                             

In [72]:
alist = ['a', 'b']
blist = ['c', 'd']
alist.extend(blist)
print(alist)

['a', 'b', 'c', 'd']


In [98]:
def clean_text(posts, tokenizer, stemmer):
    stemmer_inst = stemmer()
    tokenizer_inst = tokenizer()
    cleaned_posts = []
    for post in posts:
        cleaned_words = []
        
        #strip punctuation, digits and stopwords from whole post
        stripped_post = strip_punctuation_digits(post)
        tokenized_post = tokenizer_inst.tokenize(stripped_post)
        
        
        for word in tokenized_post:
            low_word = stemmer_inst.stem(word.lower())
            cleaned_words.append(low_word)
            #remove stopwords?
            #if low_word not in stopwords:
            #    cleaned_words.append(low_word)
        cleaned_posts.append(' '.join(cleaned_words))
    return cleaned_posts

In [74]:
stemmer = PorterStemmer
tokenizer = WhitespaceTokenizer

In [99]:
cleaned_train_posts = clean_text(train_post_texts[0:5], tokenizer, stemmer)

In [100]:
cleaned_train_posts

['im an antisjw libertarian but i run an instagram profil for my witchthem illustr given that libertarian tend to be logicori men and witch fan tend to be leftist women i often get follow i wish didnt even exist i tri includ libertarian in my bio but that didnt work onc i even post a parodi of witch where the witch held libertarian sign rather than leftist and an illiter appar sjw comment in approv it tempt to argu with these peopl or at least let them know they have the wrong idea about me but i feel like i should focu on the busi aspect of thing and shut up and bake the fuck cake also even though im not from north america or europ im paranoid about dox and such edit guy your suppos to convinc me that i should argu',
 'warn lot of runon sentenc and opinion be state as fact lot of assumpt too im readi to be call out on those thing valu and the concept of moral and ethic are inform upon by environment factor dure childhood usual a person adopt the valu of their peer and roll model note 

In [40]:
testy = """
Hi don't worry text! lots of punctuation, in this one'.
and did you consider new lines?
. best not    there we go!
""" 


In [28]:
punctuation_replacement = str.maketrans({key: None for key in string.punctuation}) 
testy.translate(punctuation_replacement)


'Hi dont worry text lots of punctuation in this one'

In [82]:
testystrip = strip_punctuation_digits(testy)

In [85]:
testystrip

'\nHi dont worry text lots of punctuation in this one\nand did you consider new lines\n best not    there we go\n'

In [90]:
toky = WhitespaceTokenizer()
toky.tokenize(text=testystrip)

['Hi',
 'dont',
 'worry',
 'text',
 'lots',
 'of',
 'punctuation',
 'in',
 'this',
 'one',
 'and',
 'did',
 'you',
 'consider',
 'new',
 'lines',
 'best',
 'not',
 'there',
 'we',
 'go']

In [87]:
import re
testy.split(' ')

['\nHi',
 "don't",
 'worry',
 'text!',
 'lots',
 'of',
 'punctuation,',
 'in',
 'this',
 "one'.\nand",
 'did',
 'you',
 'consider',
 'new',
 'lines?\n.',
 'best',
 'not',
 '',
 '',
 '',
 'there',
 'we',
 'go!\n']